# Shor's Experiemnt in quantum computer

## Step 0: Import some libraries & initialize variable

In [ ]:
# Qiskit
## Step 1
from qiskit import QuantumCircuit
## Step 2 
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# NOTE To save the api
service = QiskitRuntimeService(channel="ibm_quantum",
                              token="")
# QiskitRuntimeService.save_account(channel="ibm_quantum",
#                               token="")
## Step 3
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import transpile
## Step 4
import matplotlib.pyplot as plt

# Math
import numpy as np
import fractions
from math import gcd
from scipy.linalg import fractional_matrix_power


/var/folders/_w/jt_kf9wn083f9d6qv_cfhz500000gn/T/ipykernel_55564/723020016.py:8: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum",


In [30]:
n_target_wires = 8  # Number of qubits for the target register (4 for testing)
n_estimation_wires = 12  # Number of qubits for phase estimation (3 for testing)

## Step 1: Map the problem to the circuit

In [31]:
def qpe_circuit(matrix, n_target_wires, n_estimation_wires):
    """Construct the Quantum Phase Estimation circuit."""
    qc = QuantumCircuit(n_target_wires + n_estimation_wires, n_estimation_wires)

    # Prepare the target register in |1> state
    qc.x(0)

    # Apply Hadamard gates to the estimation qubits
    for wire in range(n_target_wires, n_target_wires + n_estimation_wires):
        qc.h(wire)

    # Apply controlled unitaries
    for idx in range(n_estimation_wires):
        power = 2 ** (n_estimation_wires - idx - 1)
        U_power = np.linalg.matrix_power(matrix, power)

        # Create a gate from the matrix and apply it as a controlled operation
        gate = QuantumCircuit(n_target_wires)
        gate.unitary(U_power, range(n_target_wires))
        controlled_gate = gate.control()
        qc.append(controlled_gate, [n_target_wires + idx] + list(range(n_target_wires)))

    # Apply the inverse Quantum Fourier Transform (QFT)
    for j in range(n_estimation_wires - 1, -1, -1):
        qc.h(n_target_wires + j)
        for m in range(j):
            angle = -np.pi / (2 ** (j - m))
            qc.cp(angle, n_target_wires + j, n_target_wires + m)
    

    # Measure the estimation qubits
    qc.measure(range(n_target_wires, n_target_wires + n_estimation_wires), range(n_estimation_wires))

    return qc

## Step 2: Optimize the problem for quantum execution

In [32]:
backend_name = "ibm_sherbrooke"
backend = service.backend(backend_name)
sampler = Sampler(mode=backend)
sampler.options.default_shots = 100

## Step 3: Execute on the backend

In [ ]:
def get_phase(qc, sampler, n_estimation_wires):
    """Estimate the phase from the QPE circuit using the Runtime Sampler."""
    job = sampler.run([(qc, [])])
    result = job.result()
    pub_result = result[0]
    counts = pub_result.join_data().get_counts()
    most_likely_bitstring = max(counts, key=counts.get)
    decimal = int(most_likely_bitstring, 2)
    phase = decimal / (2 ** n_estimation_wires)
    return phase

def get_period(matrix, backend, sampler, n_target_wires, n_estimation_wires):
    """Estimate the period using multiple runs of phase estimation."""
    periods = []
    qc = qpe_circuit(matrix, n_target_wires, n_estimation_wires)
    transpiled_qc = transpile(qc, backend=backend, optimization_level=1)
    phase = get_phase(transpiled_qc, sampler, n_estimation_wires)
    fraction = fractions.Fraction(phase).limit_denominator(2 ** n_estimation_wires)
    periods.append(fraction.denominator)
    
    return max(periods)

def get_matrix_a_mod_N(a, N):
    """Build the unitary matrix for modular multiplication a mod N."""
    dim = 2 ** n_target_wires
    U = np.zeros((dim, dim), dtype=complex)
    for x in range(dim):
        if x < N:
            y = (a * x) % N
        else:
            y = x
        U[y, x] = 1
    return U

## Step 3.2: Actual Algorithm

In [40]:
# Utility functions
def is_coprime(a, N):
    """Check if a and N are coprime."""
    return gcd(a, N) == 1


def is_odd(r):
    """Check if the period r is odd."""
    return r % 2 == 1


def is_not_one(x, N):
    """Check if x is not congruent to 1 modulo N."""
    return (x % N) != 1


In [41]:
def shor(N):
    """Run Shor's algorithm to find factors of N."""
    for a in range(2, N - 1):
        if not is_coprime(a, N):
            p = gcd(a, N)
            q = N // p
            break
        else:
            U_na = get_matrix_a_mod_N(a, N)
            r = get_period(U_na, backend, sampler, n_target_wires, n_estimation_wires)
            if not is_odd(r):
                x = pow(a, r // 2, N)
                if is_not_one(x, N):
                    p = gcd(x - 1, N)
                    q = gcd(x + 1, N)
                    if p * q == N and p != 1 and q != 1:
                        break
    return [p, q]

In [42]:
# Run the algorithm CRITICAL AGAIN WILL LOSE YOUR MONEY
test_numbers = [
    15,  # 3 * 5
    15,  # 3 * 5
    15,  # 3 * 5
    21,  # 3 * 7
    21,  # 3 * 7
    21,  # 3 * 7
]

for N in test_numbers:
    print("N: ", N)
    factors = shor(N)
    print("Factors of", N, "are:", factors)

N:  15
Factors of 15 are: [3, 5]
N:  15
Factors of 15 are: [3, 5]
N:  15
Factors of 15 are: [3, 5]
N:  21
Factors of 21 are: [3, 7]
N:  21
Factors of 21 are: [3, 7]
N:  21
Factors of 21 are: [3, 7]


## Step 4: Analyze the result